In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from werkzeug.security import generate_password_hash, check_password_hash
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, login_required, logout_user, current_user, LoginManager
from flask_wtf import FlaskForm
from flask_bcrypt import Bcrypt
from flask_admin import Admin
from flask_admin.contrib.sqla import ModelView
from datetime import datetime

app = Flask(__name__)
app.config['FLASK_ADMIN_SWATCH'] = 'cerulean'
admin = Admin(app, name='管理员', template_mode='bootstrap3')
app.config['SECRET_KEY'] = 'thisissecretkey'

db = SQLAlchemy(app)#create the database instance
bcrypt = Bcrypt(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///demoapp.db'#connect app files to database? Not sure

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)

class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50),unique=True)
    email = db.Column(db.String(50),unique=True)
    password = db.Column(db.String(30))
    is_admin=db.Column(db.Integer)
    
    def __repr__(self):
        return '<User %r>' % self.id

admin.add_view(ModelView(User, db.session))

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    content = db.Column(db.String(200), nullable=False)
    date_expected = db.Column(db.DateTime, nullable=False)

    def __repr__(self):
        return '<Task %r>' % self.id
    
admin.add_view(ModelView(Todo, db.session))
    
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/admin")
def admin():
    return render_template("admin/index.html")

@app.route("/blogs")
def blogs():
    df = pd.read_csv("shopping_info.csv")
    list = df.to_dict('records')
    return render_template('blogs.html',entries = list)

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/post")
def post():
    return render_template('post.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.route("/dashboard")
def dashboard():
    return render_template('dashboard.html')

@app.route("/register", methods=["GET", "POST"])#这里注册
def register():
    if request.method == "POST":
        username = request.form['username']#后面是页面输入框表单名字  前面是变量
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = User(username=username,email=email,password=password,is_admin=0) #利用临时变量的值传入__init.py   
        try:
            db.session.add(new_user)
            db.session.commit()#上传
            print("Registered!")
            return redirect(url_for('register'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('register.html')

@app.route('/login', methods=["GET", "POST"])  # Login Here!
def login():
    if request.method == "POST":
        # Retrieve information
        inputted_email = request.form['email']
        inputted_password = request.form['password']

        # Send a query to db to retrieve user
        retrievedUser = db.session.query(User).filter_by(email=inputted_email).first()

        if not retrievedUser:  # User not found
            print("This User does not exist")
            flash("Wrong username or username not found.")
            return render_template('login.html')

        correct_password = generate_password_hash(retrievedUser.password)
        if check_password_hash(correct_password, inputted_password):  # Password correct
            print("Login Success!")
            flash("Successfully logged in.")
            if retrievedUser.is_admin==1:
                return redirect(url_for('admin'))
            return redirect(url_for('index'))
        else:  # Password incorrect
            print("Incorrect password.")
            flash("Wrong password.")
            return render_template('login.html')

    else:
        return render_template('login.html')
    
@app.route("/logout")
def logout():
    print('logout user.')
    flash('logged out.', current_user.username)
    logout_user()
    redirect(url_for('index'))
    return redirect(url_for('index'))   



    
#########################################################################################

if __name__ == "__main__":
    app.run('localhost', 3350)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/sophiawang/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/Users/sophiawang/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
/Users/sophiawang/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:3350/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2021 18:04:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - -

Login Success!


127.0.0.1 - - [01/Nov/2021 18:04:15] "GET /admin/todo/ HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 18:04:17] "GET /admin/todo/new/?url=%2Fadmin%2Ftodo%2F HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 18:04:25] "POST /admin/todo/new/?url=%2Fadmin%2Ftodo%2F HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 18:04:25] "GET /admin/todo/ HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 18:04:47] "GET /admin/todo/new/?url=%2Fadmin%2Ftodo%2F HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 18:04:52] "POST /admin/todo/new/?url=%2Fadmin%2Ftodo%2F HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 18:04:52] "GET /admin/todo/ HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 18:04:54] "GET /admin/todo/new/?url=%2Fadmin%2Ftodo%2F HTTP/1.1" 200 -


In [2]:
import sqlite3 
con = sqlite3.connect('demoapp.db')
cursor = con.cursor()
sql = "CREATE TABLE Todo (id INTEGER PRIMARY KEY AUTOINCREMENT, content CHAR(200), date_expected DATE);"
try:
    cursor.execute(sql)
    con.commit()
    print("Table successfully created.")
except Exception as e:
    print("Exception occurred.",e)
cursor.close()
con.close()

Table successfully created.
